# Topic analysis (LDA)

In [26]:
from pprint import pprint
import json
import pandas as pd  
import numpy as np
import re

Data uitlezen. Eerst getest met een dataset van 1000 tweets. Daarna mini_twitter_geotagged_clean. Gebruik de hele dataset later.

In [63]:
%%time
# Verander in de regel hieronder de locatie van de data
tweets = pd.read_json("mini_twitter_geotagged_clean.json", orient='records', lines=True)

Wall time: 18.3 s


In [64]:
tweets['CleanText'] = tweets['CleanText'].replace('\n','')
tweets['CleanText'] = tweets['CleanText'].replace('\t','')
tweets['CleanText'] = tweets['CleanText'].replace('amp','')
tweets['CleanText'] = tweets['CleanText'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [65]:
tweets.head()

CleanText  contributors  \
0              COLLUSION TOGETHER httpstco5GMNZq40V3           NaN   
1            will year things should have done eight           NaN   
2  clear deliberately throwing this racein 2007 k...           NaN   
3  wouldnt recognize came from your mouth they co...           NaN   
4  know suing someone Thats most beautiful thing ...           NaN   

  coordinates          created_at  \
0        None 2016-08-12 10:04:02   
1        None 2016-08-12 10:04:21   
2        None 2016-08-12 10:04:30   
3        None 2016-08-12 10:04:46   
4        None 2016-08-12 10:04:48   

                                            entities extended_entities  \
0  {'hashtags': [{'text': 'NOJUSTICE', 'indices':...              None   
1  {'hashtags': [], 'urls': [], 'user_mentions': ...              None   
2  {'hashtags': [{'text': 'CNN', 'indices': [0, 4...              None   
3  {'hashtags': [{'text': 'NeverTrump', 'indices'...              None   
4  {'hashtags': [], 'urls': [], 'user_mentions': ...              None   

   favorite_count  favorited filter_level   geo  \
0               0      False          low  None   
1               0      False          low  None   
2               0      False          low  None   
3               0      False          low  None   
4               0      False          low  None   

                         ...                          \
0                        ...                           
1                        ...                           
2                        ...                           
3                        ...                           
4                        ...                           

                                       quoted_status  quoted_status_id  \
0                                               None               NaN   
1                                               None               NaN   
2                                               None               NaN   
3  {'created_at': 'Fri Aug 12 04:10:25 +0000 2016...      7.639507e+17   
4                                               None               NaN   

  quoted_status_id_str  retweet_count  retweeted  \
0                  NaN              0      False   
1                  NaN              0      False   
2                  NaN              0      False   
3         7.639507e+17              0      False   
4                  NaN              0      False   

                                              source  \
0  <a href="http://twitter.com/download/android" ...   
1  <a href="http://twitter.com/download/iphone" r...   
2  <a href="http://twitter.com" rel="nofollow">Tw...   
3  <a href="http://twitter.com/download/iphone" r...   
4  <a href="http://twitter.com/download/iphone" r...   

                                                text            timestamp_ms  \
0  @BarackObama \n@FBI\n@LORETTALYNCH \nALL IN CO... 2016-08-12 10:04:02.194   
1  @HillaryClinton he will do in one year all the... 2016-08-12 10:04:21.125   
2  #CNN #newday clear #Trump deliberately throwin... 2016-08-12 10:04:30.035   
3  @realDonaldTrump, you wouldn't recognize a lie... 2016-08-12 10:04:46.265   
4  "Kid, you know, suing someone? Thats the most ... 2016-08-12 10:04:48.229   

  truncated                                               user  
0     False  {'id': 82496193, 'id_str': '82496193', 'name':...  
1     False  {'id': 44032624, 'id_str': '44032624', 'name':...  
2     False  {'id': 769208504, 'id_str': '769208504', 'name...  
3     False  {'id': 41043316, 'id_str': '41043316', 'name':...  
4     False  {'id': 762090248159371264, 'id_str': '76209024...  

[5 rows x 31 columns]

we maken een array aan met alle clean tekst uit de tweets

In [67]:
doc_complete = tweets['CleanText'].values

array(['COLLUSION TOGETHER httpstco5GMNZq40V3',
       'will year things should have done eight',
       'clear deliberately throwing this racein 2007 knew that destabilization Mideast started wIraq invasion',
       ..., 'clear want them write what want sorry Donny',
       'picked staring straight into face base your party pretty NeverTrump',
       'Vote Jill Stein Trump wins 4yra Trumps hell better than stopping Hillary'],
      dtype=object)


We moeten nog lemmatizen en stopwoorden verwijderen

In [68]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

# Create a set of stopwords
stop = set(stopwords.words('english'))

# Create a set of punctuation words 
exclude = set(string.punctuation) 

# This is the function makeing the lemmatization
lemma = WordNetLemmatizer()

# In this function we perform the entire cleaning
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

# This is the clean corpus.
doc_clean = [clean(doc).split() for doc in doc_complete] 

Maak een dictionary aan voor alle woorden

In [69]:
# Importing Gensim
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

LDA toepassen
pas de num_topics aan voor het aantal topics: getest op een paar topics tot 100. Meer geeft een duidelijkere groepering, moeten nog kijken wat we kunnen gebruiken voor het verslag.
Hoe hoger het aantal passes hoe beter het resultaat. Passes getest op 50. Dat duurt ong 15 minuten op mijn laptop.

In [70]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=20, id2word = dictionary, passes=50)

nu kunnen we de topics printen
zorg dat num_topics hetzelfde is als hierboven.
num_words laat het aantal woorden zien per topic. 

In [71]:
# Print 2 topics and describe then with 4 words.
topics = ldamodel.print_topics(num_topics=20, num_words=5)

i=0
for topic in topics:
    print ("Topic",i ,"->", topic)     
    i+=1


Topic 0 -> (0, '0.046*"obama" + 0.042*"show" + 0.040*"return" + 0.037*"tax" + 0.036*"trump"')
Topic 1 -> (1, '0.032*"woman" + 0.026*"point" + 0.024*"someone" + 0.020*"potus" + 0.018*"else"')
Topic 2 -> (2, '0.080*"medium" + 0.043*"never" + 0.029*"truth" + 0.026*"news" + 0.025*"trump"')
Topic 3 -> (3, '0.068*"youre" + 0.059*"donald" + 0.043*"trump" + 0.028*"candidate" + 0.022*"talking"')
Topic 4 -> (4, '0.050*"know" + 0.035*"doesnt" + 0.034*"cant" + 0.032*"president" + 0.031*"word"')
Topic 5 -> (5, '0.031*"republican" + 0.030*"trump" + 0.028*"sure" + 0.028*"still" + 0.025*"hell"')
Topic 6 -> (6, '0.037*"back" + 0.032*"wont" + 0.030*"talk" + 0.028*"maybe" + 0.021*"job"')
Topic 7 -> (7, '0.036*"press" + 0.033*"ever" + 0.024*"always" + 0.021*"crowd" + 0.021*"liberal"')
Topic 8 -> (8, '0.079*"please" + 0.059*"maga" + 0.051*"stop" + 0.043*"help" + 0.041*"nevertrump"')
Topic 9 -> (9, '0.044*"election" + 0.036*"campaign" + 0.025*"message" + 0.023*"money" + 0.021*"done"')
Topic 10 -> (10, '0.02